In [ ]:
# %load ../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from datetime import datetime
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC, NWTG
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
class PS_Test(holo.librarian._Param_Space):
    
    def __init__(self, log, nsamples, sam_shape, seed):
        super().__init__(
            log, nsamples, sam_shape, seed,
            par1=holo.librarian.PD_Uniform(-1.0, 1.0),
            par2=holo.librarian.PD_Uniform_Log(10.0, 1000.0),
            par3=holo.librarian.PD_Normal(0.0, 4.0),
        )
        return
    
    def model_for_number(self, num):
        # CONSTRUCT SAM and HARD models here
        sam = None
        hard = None
        return sam, hard
    
nsamps = 1000
test = PS_Test(log, nsamps, 50, 12345)

for ii in np.random.choice(nsamps, 4, replace=False):
    print(f"\ntest sample {ii:4d} :: {test.params(ii)}  \n    {test.param_dict(ii)}")

fig, axes = plt.subplots(figsize=[10, 5], ncols=test.ndims)
for ii, ax in enumerate(axes):
    ax.grid(True, alpha=0.25)
    kale.dist1d(test._params[:, ii], ax=ax, density=False)
    
plt.show()


# LinLog

In [ ]:
test = holo.librarian.PD_Lin_Log(0.01, 100.0, 0.1, 0.5)
xx = np.linspace(0.0, 1.0, 10000)
yy = test(xx)
print(utils.minmax(yy))
plt.loglog(xx, yy)
ax = plt.gca()
ax.axhline(test._crit, color='r', ls=':')
ax.axvline(test._lofrac, color='r', ls=':')
plt.show()

In [ ]:
NUM = int(1e4)
crit = 0.1

BINS = 20
e1 = np.linspace(0.01, crit, BINS, endpoint=False)
e2 = np.logspace(*np.log10([crit, 100.0]), BINS)
edges = np.concatenate([e1, e2])

fig, ax = plot.figax(scale='log')
for frac in [0.2, 0.5, 0.8]:
    test = PD_Lin_Log(0.01, 100.0, crit, frac)
    xx = test(np.random.uniform(0.0, 1.0, size=NUM))
    kale.dist1d(xx, ax=ax, edges=edges, density=True, probability=False)
    obs_frac = np.count_nonzero(xx < crit) / xx.size
    print(f"target:{frac:.2f}, result:{obs_frac:.4f}", 1.0/np.sqrt(NUM))
    assert np.isclose(frac, obs_frac, atol=2.0/np.sqrt(NUM))
    
plt.show()

In [ ]:
NUM = int(1e4)
frac = 0.5

BINS = 20
# e1 = np.linspace(0.01, crit, BINS, endpoint=False)
# e2 = np.logspace(*np.log10([crit, 100.0]), BINS)
# edges = np.concatenate([e1, e2])
edges = np.logspace(*np.log10([0.01, 100.0]), 2*BINS)

fig, ax = plot.figax(scale='log')
for crit in [0.1, 1.0, 10.0]:
    test = PD_Lin_Log(0.01, 100.0, crit, frac)
    xx = test(np.random.uniform(0.0, 1.0, size=NUM))
    kale.dist1d(xx, ax=ax, edges=edges, density=True, probability=False)
    obs_frac = np.count_nonzero(xx < crit) / xx.size
    print(f"target:{frac:.2f}, result:{obs_frac:.4f}", 1.0/np.sqrt(NUM))
    assert np.isclose(frac, obs_frac, atol=2.0/np.sqrt(NUM))
    
plt.show()

# LogLin

In [ ]:
class PD_Log_Lin(holo.librarian._Param_Dist):

    def __init__(self, lo, hi, crit, lofrac):
        self._lo = lo
        self._hi = hi
        self._crit = crit
        self._lofrac = lofrac
        return

    def _dist_func(self, xx):
        lo = self._lo
        hi = self._hi
        crit = self._crit
        lofrac = self._lofrac
        xx = np.atleast_1d(xx)
        yy = np.empty_like(xx)
        loidx = (xx <= lofrac)

        l10_lo = np.log10(lo)
        l10_crit = np.log10(crit)

        temp = l10_lo + (l10_crit - l10_lo) * xx[loidx] / lofrac
        yy[loidx] = np.power(10.0, temp)

        hiidx = ~loidx
        yy[hiidx] = crit + (hi - crit) * (xx[hiidx] - crit) / (1.0 - crit)
        return yy

In [ ]:
test = PD_Log_Lin(0.01, 100.0, 0.1, 0.5)
xx = np.linspace(0.0, 1.0, 10000)
yy = test(xx)
print(utils.minmax(yy))
plt.loglog(xx, yy)
ax = plt.gca()
ax.axhline(test._crit, color='r', ls=':')
ax.axvline(test._lofrac, color='r', ls=':')
plt.show()